# Spark SQL Quiz

This quiz uses the same dataset and questions from the Spark Data Frames Programming Quiz. For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql import types
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [2]:
spark = SparkSession\
        .builder\
        .appName("My Spark SQL Quiz")\
        .getOrCreate()

In [4]:
log_data = spark.read.json("data/sparkify_log_small.json")
log_data.count()

10000

# Question 1

Which page did user id ""(empty string) NOT visit?

In [5]:
log_data.createOrReplaceTempView("log_data_table")

In [6]:
spark.sql("""
    SELECT DISTINCT page
    FROM log_data_table
    WHERE page NOT IN (SELECT DISTINCT page
                        FROM log_data_table
                        WHERE userId == '')
""").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [7]:
spark.sql("""
    SELECT DISTINCT page
    FROM log_data_table
    WHERE page NOT IN (SELECT DISTINCT page
                        FROM log_data_table
                        WHERE userId == '')
""").explain()

== Physical Plan ==
*(5) HashAggregate(keys=[page#16], functions=[])
+- Exchange hashpartitioning(page#16, 200)
   +- *(4) HashAggregate(keys=[page#16], functions=[])
      +- *(4) Project [page#16]
         +- BroadcastNestedLoopJoin BuildRight, LeftAnti, ((page#16 = page#16#77) || isnull((page#16 = page#16#77)))
            :- *(1) FileScan json [page#16] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/home/workspace/data/sparkify_log_small.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<page:string>
            +- BroadcastExchange IdentityBroadcastMode
               +- *(3) HashAggregate(keys=[page#16], functions=[])
                  +- Exchange hashpartitioning(page#16, 200)
                     +- *(2) HashAggregate(keys=[page#16], functions=[])
                        +- *(2) Project [page#16]
                           +- *(2) Filter (isnotnull(userId#23) && (userId#23 = ))
                              +- *(2) FileScan json [page#16,userId#

# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?
Team may already be familiar or even skilled in SQL than Data Frames / Pandas. Some operations are just easier to do with SQL.. especially operations requiring JOINing two data sets
Data Frames - Offer more flexibility and allows for imperative programming. Shareable and increase in adaptation by Data Science community

# Question 3

How many female users do we have in the data set?

In [8]:
spark.sql("""
    SELECT COUNT(DISTINCT userId)
    FROM log_data_table
    WHERE gender == 'F'
""").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [9]:
spark.sql("""
    SELECT artist, count(*) as num_songs
    FROM log_data_table
    WHERE page == 'NextSong'
    GROUP BY artist
    ORDER BY num_songs DESC
""").show(1)

+--------+---------+
|  artist|num_songs|
+--------+---------+
|Coldplay|       83|
+--------+---------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [17]:
songs_by_user_phase = spark.sql("""
    SELECT userId, ts, song_play, 
    SUM(homepage_visit) OVER (PARTITION BY userId ORDER BY int(ts) DESC RANGE UNBOUNDED PRECEDING) as phase
    FROM (
        SELECT userId, ts,
        CASE WHEN page == 'Home' THEN 1 ELSE 0 END as homepage_visit,
        CASE WHEN page == 'NextSong' THEN 1 ELSE 0 END as song_play
        FROM log_data_table
        WHERE page == 'NextSong' OR page == 'Home'
        ) sub
""")

In [19]:
songs_by_user_phase.createOrReplaceTempView("songs_by_user_phase_table")
spark.sql("""
    SELECT AVG(num_songs) AS avg1, AVG(CASE WHEN num_songs == 0 THEN NULL ELSE num_songs END) AS avg2
    FROM (
        SELECT userId, phase, SUM(song_play) AS num_songs
        FROM songs_by_user_phase_table
        GROUP BY userId, phase
        ) sub
""").show()

+-----------------+-----------------+
|             avg1|             avg2|
+-----------------+-----------------+
|4.432819968135953|6.898347107438017|
+-----------------+-----------------+

